In [51]:
"""
观察sensor arrangement对FMG信号的影响，用label.txt的时间戳划分出包含5个完整收缩-舒张过程的信号，方便进行信号相似性分析
"""
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pytz
import time
import sys
sys.path.append("..")
from utils.utils_iFEMG_feature import read_label


folder_path = r"D:\LEARNINNNNNNNNNNNNG\ExperimentData\20250319major_revision\传感器位置对FMG的影响"
file_name_lst = ['cen-1kg', 'cen-2kg', 'nei-1kg', 'nei-2kg', 'wai-1kg', 'wai-2kg', 'jin-1kg', 'jin-2kg', 'yuan-1kg', 'yuan-2kg']


# 函数定义
def str_to_milliseconds(time_str, format_str="%Y-%m-%d %H:%M:%S,%f", timezone="UTC"):
    """
    将时间字符串转换为毫秒级时间戳
    
    参数:
        time_str: 时间字符串
        format_str: 时间格式
        timezone: 时区名称（如 'Asia/Shanghai'）
    """
    # 解析时间字符串
    dt = datetime.strptime(time_str, format_str)
    
    # 如果指定了时区，将时间转换到该时区
    if timezone:
        tz = pytz.timezone(timezone)
        dt = tz.localize(dt)
    
    # 转换为毫秒级时间戳
    return int(dt.timestamp() * 1000)

# 使用示例
time_str = "2025-03-21 14:53:24,245"

# 默认使用本地时区
local_timestamp = str_to_milliseconds(time_str)
print(f"本地时间戳: {local_timestamp}")

# 指定上海时区
shanghai_timestamp = str_to_milliseconds(time_str, timezone='Asia/Shanghai')
print(f"上海时区时间戳: {shanghai_timestamp}")

# 使用UTC时区
utc_timestamp = str_to_milliseconds(time_str, timezone='UTC')
print(f"UTC时间戳: {utc_timestamp}")

本地时间戳: 1742568804245
上海时区时间戳: 1742540004245
UTC时间戳: 1742568804245


In [52]:
filtered_signal_lst = []
for file_name in file_name_lst:
    # 文件读取
    raw_data = pd.read_table(folder_path + "\\" + file_name + ".db", sep = ';', header = None)
    label = read_label(folder_path + "\\" + file_name + ".txt")

    # 处理label中的时间戳获得信号的开始时间和结束时间
    start_t = label[0][0] + " " + label[0][1]
    start_t_stamp = str_to_milliseconds(start_t, format_str="%Y-%m-%d %H:%M:%S.%f", timezone='UTC')

    end_t = label[10][0] + " " + label[10][1]
    end_t_stamp = str_to_milliseconds(end_t, format_str="%Y-%m-%d %H:%M:%S.%f", timezone='UTC')

    # 原始信号数据中的时间字符串转换为ms精度时间戳
    raw_data['timestamp'] = pd.to_datetime(raw_data[0], format="%Y-%m-%d %H:%M:%S,%f").astype('int64') // 10**6
    
    # 根据时间戳筛选数据，channel 1是FMG信号数据，将series重命名为file_name保存在lst中，方便之后进行合并
    filtered_signal_lst.append(raw_data[(raw_data['timestamp'] >= start_t_stamp) & (raw_data['timestamp'] <= end_t_stamp)][1].rename(file_name))
    
    

In [53]:
for i in filtered_signal_lst:
    print(len(i))

113416
113256
113188
113363
112993
113166
113264
113298
113002
113568
